In [2]:
import numpy as np
import pandas as pd

In [9]:
__author__ = 'Chris Potts'


class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        The temperature parameter. Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        return rownorm(self.lexicon * self.prior)

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        spk = self.speaker().T
        return rownorm(spk * self.prior)


def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T


def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)


if __name__ == '__main__':
    """Examples"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['tailed axe', 'big axe', 'small axe', 'reversed-blade axe']
    states = ['R', 'P', 'p', 'ь']
    lex = pd.DataFrame([
        [1.0, 0.0, 0.0, 0.0],
        [1.0, 1.0, 0.0, 0.0], 
        [0.0, 0.0, 1.0, 1.0],
        [0.0, 0.0, 0.0, 1.0]], index=msgs, columns=states)

In [10]:
print("="*70 + "\nEven priors and all-0 message costs\n")
basic_mod = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.0, 0.0, 0.0, 0.0])

display_reference_game(basic_mod)

print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

Even priors and all-0 message costs



,R,P,p,ь,costs
tailed axe,1.00,0,0,0,0
big axe,1.00,1,0,0,0
small axe,0.00,0,1,1,0
reversed-blade axe,0.00,0,0,1,0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Literal listener


,R,P,p,ь
tailed axe,1.0,0.0,0.0,0.0
big axe,0.5,0.5,0.0,0.0
small axe,0.0,0.0,0.5,0.5
reversed-blade axe,0.0,0.0,0.0,1.0



Pragmatic speaker


,tailed axe,big axe,small axe,reversed-blade axe
R,0.666667,0.333333,0.000000,0.000000
P,0.000000,1.000000,0.000000,0.000000
p,0.000000,0.000000,1.000000,0.000000
ь,0.000000,0.000000,0.333333,0.666667



Pragmatic listener


,R,P,p,ь
tailed axe,1.00,0.00,0.00,0.00
big axe,0.25,0.75,0.00,0.00
small axe,0.00,0.00,0.75,0.25
reversed-blade axe,0.00,0.00,0.00,1.00


In [ ]:
big axe и small axe являются скалярными импликатурами, так как R и ь были бы определены по уникальному признаку